In [1]:
import matplotlib.pyplot as plt
from scipy.stats import binom
import numpy as np
import math
import scipy.stats as stats
from scipy.special import logsumexp
import random
import pandas as pd
import hedgehog as hh
from IPython.display import Markdown as md


def hide_code_in_slideshow():
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)

#  a hack to hide code from cell: https://github.com/damianavila/RISE/issues/32

In [2]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
</style>

In [3]:
from IPython.core.display import HTML

HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# CS5010 Artificial Intelligence Principles
### Lecture 13 Uncertainty 4
#### More on Bayesian (Belief) Networks 

Lei Fang

University of St Andrews

# Last time


* Bayesian networks
* How to construct a BN

# This time

Exact inference algorithm for a BN: enumeration algorithm

Software package for Bayesian Networks: a Python package (hedgehog https://github.com/MaxHalford/hedgehog)
  * create a BN
  * make inference 

More examples on Bayesian networks
  * walk through of Bayesian network for Sally Clark's case


# Inference based on Baye's net

<center><img src="./figs/burglar.png" width = "400" height="400"/></center>

In general, we can do two kinds of inference:
* bottom-up: given evidence (observations, data) infer the cause; opposite the direction of edges, 
  * Burglary example: $P(B|J,M)$

* top-down: given cause infer the observations (follow the direction of edge), 
  * prediction of future data
  * Burglary example: $P(J|B, E)$

# Exact inference based on Baye's net

Some top-down inference simply is reading off entries from the CPTs

<center><img src="./figs/burglarCPTs.png" width = "600" height="400"/></center>

e.g. $P(A=T|B=T, E=F), P(J|A=T), P(M|A=F)$

For others, we need to apply probability rules on the Baye's network
  * with the help of BN, automated algorithmic inference can be implemented
  * from the algorithm's perspective, bottom-up and top-down are the same 

# Top down case: $P(J|B,E)$

<center><img src="./figs/burglarCPTs.png" width = "600" height="400"/></center>

$P(J|B,E) = \alpha P(J,B,E)\;\;\; \text{conditional probability rule}$

$= \alpha \sum_{a}\sum_{m} P(B,E,A=a,J,M=m)\;\;\; \text{summation rule}$

$= \alpha \sum_{a}\sum_{m} P(B)P(E)P(A=a|B,E)P(J|A=a)P(M=m|A=a)\;\;\; \text{factoring property of BN!}$ 

A naive algorithm actually can be implemented at this stage: 
  * 5-3=2 nested sums (or loops): 5 is the total number of nodes, 3 is the size of ``query`` $\cup$ ``Evidence``={B,E,J}, 2 size of nuiance r.v.s {A, M} in this case
    * in general, the nested summation depends on the inference: i.e. how many nuiance r.v.s to sum-out
  * this week's tutorial questioin is on this naive algorithm (we resort to recursion or backtracking)

However, we can simplify the computation further 

Remember $\sum_{i=1}^n ax_i =ax_1 + a x_2 + \ldots + ax_n= a(\sum_{i=1}^n x_i)$
  * $a$ is a common factor can be taken out (saves some computation) or equivalently pushing summation $\Sigma$ inwards

$P(B)P(E)$ above is a common factor
  $$P(J|B,E)= \alpha \sum_{a}\sum_{m} P(B)P(E)P(A=a|B,E)P(J|A=a)P(M=m|A=a) \\
  = \alpha P(B)P(E)\sum_{a}\sum_{m}P(A=a|B,E)P(J|A=a)P(M=m|A=a)$$

$P(A=a|B,E)P(J|a)$ together is also a common factor (from $m$'s perspective), so we push $\Sigma_m$ inwards
  $$P(J|B,E)
  = \alpha P(B)P(E)\sum_{a}P(A=a|B,E)P(J|A=a)\sum_{m}P(M=m|A=a)$$

Finally note that conditional distributions are distributions
$$\sum_{m}P(M=m|a)=1$$

$$\Rightarrow P(J|B,E)
  = \alpha P(B)P(E)\sum_{a}P(A=a|B,E)P(J|A=a)\underbrace{\sum_{m}P(M=m|A=a)}_{1}$$

So we have 
$$P(J|B,E)
  = \alpha P(B)P(E)\sum_{a}P(A=a|B,E)P(J|A=a)$$
  * 2 nested sums to 1 summation
  * the algorithm is called: enumeration algorithm
    * enumerate all possible scenarios of the other nuiance r.v.s (sum rule)

<center><img src="./figs/burglarCPTs.png" width = "600" height="400"/></center>

To find $P(J|B=t, E=f)$, for $J=t$: $$P(J=t|B=t,E=f) = \alpha P(B=t)P(E=f)\sum_{a}P(A=a|B=t,E=f)P(J=t|A=a)\\
= \alpha* .001 * (1-.002) * (.94* .9 + (1-.94)*.05) = \alpha 0.0008473$$

For $J=f$: $$P(J=f|B=t, E=f) = \alpha P(B=t)P(E=f)\sum_{a}P(A=a|B=t,E=f)P(J=f|A=a)\\
= \alpha* .001 * (1-.002) * (.94* (1-.9) + (1-.94)*(1-.05)) = \alpha 0.0001507$$

Normalise to find $\alpha = \frac{1}{0.0008473+ 0.0001507}$
$$P(J|B=t, E=f) = \begin{cases} 0.849 & True \\ 0.151 &False \end{cases}$$

# Bottom-up inference with example $P(B|J,M)$

$$\begin{align}P(B|J,M) &= \alpha P(B,J,M)\;\;\; \text{conditional probability rule} \\
&= \alpha \sum_{e}\sum_{a} P(B,e,a,J,M)\;\;\; \text{summation rule} \\
&= \alpha \sum_{e}\sum_{a} P(B)P(e)P(a|B,e)P(J|a)P(M|a) \;\;\; \text{factoring CPTs} \\
&= \alpha P(B)\sum_{e}P(e)\sum_{a} P(a|B,e)P(J|a)P(M|a)\;\;\; \text{simplification} \end{align}$$

For the inference $P(B|J=t,M=t)$:

If sub-in the numbers and run the enumeration algorithm, 

$$P(B|J=t, M=t) = \begin{cases} 0.284 & true \\ 0.716 &false \end{cases}$$

Both top-down and bottom-up inference: enumerating and sum-out all other nuiance r.v.s

For this example, we have 2 loops (summations), $\sum_e$ and $\sum_a$
  * the `ENUMERATION-ASK` in the book implements the algorithm

# Software packages for Bayesian networks

A lot of software packages for Bayesian networks

Some of them are commercial (so we will not cover them here)
  * probably better user interface 
  * but also less freedom to customise (useless for research) 
  
A wide range of free open-source and general purpose packages available
  * Stan: MCMC (NUTS sampler) and variational inference
  * PyMC3, JAGS, OpenBUGS: MCMC only

We will use hedgehog (https://github.com/MaxHalford/hedgehog) as an example
  * simple to use and good for toy examples
  * not flexible enough for complicated models

# Installation of hedgehog

The installation steps are detailed in the package's Github website

Assume `pip` are installed (`pip3` on school server) 

`$ pip install git+https://github.com/MaxHalford/vose`

`$ pip install git+https://github.com/MaxHalford/hedgehog`


After installation, import the package 
`import hedgehog as hh`


You also need the following packages
  * ``panda``: ``import pandas as pd``

# A toy example: coin tossing

Your friend has two coins, one fair and the other bent (with $p_2=0.2$). Your friend randomly picks one and flipping that coin three times and records the result. 

The random variables: `CoinChoice`, `Y1`, `Y2`, and `Y3`

The Bayesian network is:

<center><img src="./figs/coinBN.png" width = "1000" height="500"/></center>

# Translate the BN to BayesNet object in Python


Step 1: create a BayesNet object with the four random variables
  * it also tells `Coin Choice` is the parent of `Y1`, `Y2` and `Y3`


```python
diceExamplebn = hh.BayesNet(('Coin Choice', 'Y1') ,('Coin Choice', 'Y2'), ('Coin Choice' , 'Y3'))
```


or equivalently, use list `[]`: 

```python
diceExamplebn = hh.BayesNet(('Coin Choice', ['Y1', 'Y2', 'Y3']))
```

Step 2: add CPTs for each node


```python
diceExamplebn.P['Coin Choice'] = pd.Series({'Coin 1': 0.5, 'Coin 2': 0.5})

diceExamplebn.P['Y1'] = pd.Series({
    ('Coin 1', 'Head') : 0.5,
    ('Coin 1', 'Tail') : 0.5,
    ('Coin 2', 'Head') : 0.2,
    ('Coin 2', 'Tail') : 0.8
})
...
```

Step 3 `prepare` the network 

```python
diceExamplebn.prepare()
```

You may also plot the network object to verify visually

```python
dot = diceExamplebn.graphviz()
path = dot.render('diceExample', directory='figs', format='png', cleanup=True)
```

<center><img src="./figs/coinExample.png" width = "400" height="500"/></center>

In [4]:
diceExamplebn = hh.BayesNet(('Coin Choice', ['Y1', 'Y2', 'Y3']))
diceExamplebn.P['Coin Choice'] = pd.Series({'Coin 1': 0.5, 'Coin 2': 0.5})
diceExamplebn.P['Y1'] = pd.Series({
    ('Coin 1', 'Head'): 0.5,
    ('Coin 1', 'Tail'): 0.5,
    ('Coin 2', 'Head'): 0.2,
    ('Coin 2', 'Tail'): 0.8
})

diceExamplebn.P['Y2'] = pd.Series({
    ('Coin 1', 'Head'): 0.5,
    ('Coin 1', 'Tail'): 0.5,
    ('Coin 2', 'Head'): 0.2,
    ('Coin 2', 'Tail'): 0.8
})

diceExamplebn.P['Y3'] = pd.Series({
    ('Coin 1', 'Head'): 0.5,
    ('Coin 1', 'Tail'): 0.5,
    ('Coin 2', 'Head'): 0.2,
    ('Coin 2', 'Tail'): 0.8
})

diceExamplebn.prepare()
# dot5 = diceExamplebn.graphviz()
# path5 = dot5.render('coinExample', directory='figs', format='png', cleanup=True)

AttributeError: module 'hedgehog' has no attribute 'BayesNet'

### Last step: make probabilistic inference 

Bottom-up inference: 

$P(C|Y1, Y2, Y3)$

In [ ]:
diceExamplebn.query('Coin Choice', event={'Y1': 'Head', 'Y2': 'Head', 'Y3': 'Head'})

In [ ]:
diceExamplebn.query('Coin Choice', event={'Y1': 'Tail', 'Y2': 'Tail', 'Y3': 'Tail'})

Prediction inference: e.g. 

$P(Y_3|Y_1, Y_2)$

Remember, marginally, the tosses are not independent
  * more likely to see a Tail if previous is a Tail as the it is more likely the bent coin has been used !

Marginal prediction (conditional on nothing): 

$P(Y_3)$

In [ ]:
# top-down inference, predict data given previous data
diceExamplebn.query('Y3', event={})

Prediction based on one observation: 

$P(Y_3|Y_1=t)$

In [ ]:
diceExamplebn.query('Y3', event={'Y1': 'Tail'})

Prediction based on two previous observations: 

$P(Y_3|Y_1=t, Y_2=t)$

In [ ]:
diceExamplebn.query('Y3', event={'Y1': 'Tail', 'Y2': 'Tail'})

# Another example: Burglary



<center><img src="./figs/burglarCPTs.png" width = "600" height="400"/></center>

In [ ]:
burglarybn = hh.BayesNet(('Burglary', 'Alarm'),
                         ('Earthquake', 'Alarm'),
                         ('Alarm', 'John calls'),
                         ('Alarm', 'Mary calls'))
# P(Burglary)
burglarybn.P['Burglary'] = pd.Series({False: .999, True: .001})
# P(Earthquake)
burglarybn.P['Earthquake'] = pd.Series({False: .998, True: .002})
# P(Alarm | Burglary, Earthquake)
burglarybn.P['Alarm'] = pd.Series({
    (True, True, True): .95,
    (True, True, False): .05,

    (True, False, True): .94,
    (True, False, False): .06,

    (False, True, True): .29,
    (False, True, False): .71,

    (False, False, True): .001,
    (False, False, False): .999
})

# P(John calls | Alarm)
burglarybn.P['John calls'] = pd.Series({
    (True, True): .9,
    (True, False): .1,
    (False, True): .05,
    (False, False): .95
})

# P(Mary calls | Alarm)
burglarybn.P['Mary calls'] = pd.Series({
    (True, True): .7,
    (True, False): .3,
    (False, True): .01,
    (False, False): .99
})
burglarybn.prepare()

### Make probabilistic inferences

$P(Burglary |M =true, J=true)$

In [ ]:
burglarybn.query('Burglary', event={'Mary calls': True, 'John calls': True})

$P(J |E =false, B=true)$

In [ ]:
burglarybn.query('John calls', event={'Earthquake': False, 'Burglary': True})

# Sally Clark's case 

Problem can be summarised: Sally Clark a middle class mother whose two children died. The causes of the deaths are assumed either SIDS (a rare disease for infants) or murder. 

More details in the motivation lecture.

Step 1: identify random variables

`Child 1 death cause` $\in \{SIDS, murder\}$: cause of death for child one

`Child 2 death cause` $\in \{SIDS, murder\}$: cause of death for child two

`Findings` $\in \{either Child Muder, both Child Murder, none Murder\}$

`Guilty` $\in \{true, false\}$: i.e. Sally is guilty or not

`Sign of disease for Child 1` $\in \{true, false\}$: has sign of disease/history

`Sign of disease for Child 2` $\in \{true, false\}$: has sign of disease/history

`Injury for Child 1` $\in \{true, false\}$: has any sign of injury

`Injury for Child 2` $\in \{true, false\}$: has any sign of injury

An order: causes precede effects :

[`Child 1 Death Cause`, `Child 2 Death Cause`,  `Sign of disease C1`, `Injury C1`, `Sign of disease C2`, `Injury C2`, `Findings`, `Guilty`]

  * we already known SIDS are genetically linked and environment factor is a key
  * the death causes are not independent: Child 2 depends on Child 1

###  [`Child 1 Death Cause`, `Child 2 Death Cause`]

Consider the first two r.v.s
  * they are not independent 
  * the chance of second SIDS is 10 times higher (genetic/environment factors)

<center><img src="./figs/sallyclark1.png" width = "2500" height="3000"/></center>

CPTs for `C1` and `C2`
* P(C1 = murder) = 0.02; P(C1 = SIDS) = 0.98

* P(C2 = SIDS|C1 = SIDS) = 0.99; P(C2 = murder|C1 = SIDS) = 0.01
* P(C2 = SIDS|C1 = murder) = 0.01; P(C2 = murder|C1 = murder) = 0.99

###  [`Child 1 Death Cause`, `Child 2 Death Cause`, `Sign of disease C1`, `Injury C1`]

Add another two r.v.s `Sign of disease C1` and `Injury C1`
  * they clearly only depends on the cause of death for child 1 

<center><img src="./figs/sallyclark2.png" width = "2500" height="1000"/></center>

CPTs for `Sign of disease: C1` (D1) and `Sign of injury: C1` (I1)

CPT for D1:
* P(D1=true| C1= murder) = 0.1; P(D1=false|C1=murder) = 0.9
* P(D1=true| C1= SIDS) = 0.95; P(D1=false|C1= SIDS) = 0.05

CPT for I1 (i.e. Bruising):
* P(I1=true| C1= murder) = 0.9; P(I1=false|C1=murder) = 0.1
* P(I1=true| C1= SIDS) = 0.15; P(I1=false|C1= SIDS) = 0.85

###  [`Child 1 Death Cause`, `Child 2 Death Cause`, `Sign of disease C1`, `Injury C1`, `Sign of disease C2`, `Injury C2`]


Add another two r.v.s `Sign of disease C2` and `Injury C2`
  * they clearly only depends on the cause of death for child 2 

<center><img src="./figs/sallyclark3.png" width = "2500" height="1000"/></center>

CPTs for `Sign of disease: C1` (D1) and `Sign of injury: C1` (I1) are the same for child ones'

### [`Child 1 Death Cause`, `Child 2 Death Cause`,  `Sign of disease C1`, `Injury C1`, `Sign of disease C2`, `Injury C2`, `Guilty`]

An alternative model without $\require{cancel}\cancel{Findings}$: add `Guilty` directly
  * depends on both children's causes of deaths
  * a deterministic function a if-elseif-else is enough
    * e.g. if $C1 \neq murder\; \&\& \;C2 \neq murder$, then $P(Guilty = guilty|\ldots) = 0$ else $P(Guilty = innocent|\cdots) = 1$

<center><img src="./figs/sallyclark4.png" width = "2500" height="1000"/></center>

### [`Child 1 Death Cause`, `Child 2 Death Cause`,  `Sign of disease C1`, `Injury C1`, `Sign of disease C2`, `Injury C2`, `Findings`, `Guilty`]

With both `Findings` and `Guilty`

<center><img src="./figs/sallyclark5.png" width = "2500" height="1000"/></center>

Two deterministic transformations CPTs

`Findings`: 
  * if $C1 = murder\; \&\& \;C2 = murder$, then $P(F = bothMurder |\ldots) = 1$ 
  * elseif $C1 = \text{SIDS}\; \&\& \;C2  = \text{SIDS}$, then $P(F = neitherMurder |\ldots) = 1$ 
  * else  $P(F = eitherMurder |\ldots) = 1$ 
  
`Guilty`: another demerministic transformation based on the `Findings`  

# In Python with hedgehog

Network arthitecture 

In [ ]:
sallybn = hh.BayesNet(
    ('Child 1 Death Cause', 'Child 2 Death Cause'),
    ('Child 1 Death Cause', 'Child 1 Bruising'),
    ('Child 1 Death Cause', 'Child 1 Sign of Disease'),
    ('Child 2 Death Cause', 'Child 2 Bruising'),
    ('Child 2 Death Cause', 'Child 2 Sign of Disease'),
    (['Child 1 Death Cause', 'Child 2 Death Cause'], 'Findings'),
    ('Findings', 'Guilty'))

CPTs specifications

In [ ]:
# P('Child 1 Death Cause')
sallybn.P['Child 1 Death Cause'] = pd.Series({'SIDS': .92, 'Murder': .08})

# P(Child 2 Death Cause|'Child 1 Death Cause')
sallybn.P['Child 2 Death Cause'] = pd.Series({
    ('SIDS', 'SIDS'): 0.99,
    ('SIDS', 'Murder'): 0.01,
    ('Murder', 'SIDS'): 0.01,
    ('Murder', 'Murder'): 0.99
})

# P(Child 1 Bruising|'Child 1 Death Cause')
sallybn.P['Child 1 Bruising'] = pd.Series({
    ('SIDS', True): 0.15,
    ('SIDS', False): 0.85,
    ('Murder', True): 0.9,
    ('Murder', False): 0.1
})

# P(Child 1 Sign of Disease|'Child 1 Death Cause') 
sallybn.P['Child 1 Sign of Disease'] = pd.Series({
    ('SIDS', True): 0.95,
    ('SIDS', False): 0.05,
    ('Murder', True): 0.1,
    ('Murder', False): 0.9
})

# P(Child 2 Bruising|'Child 2 Death Cause')
sallybn.P['Child 2 Bruising'] = pd.Series({
    ('SIDS', True): 0.15,
    ('SIDS', False): 0.85,
    ('Murder', True): 0.9,
    ('Murder', False): 0.1
})

# P(Child 2 Sign of Disease|'Child 2 Death Cause')
sallybn.P['Child 2 Sign of Disease'] = pd.Series({
    ('SIDS', True): 0.95,
    ('SIDS', False): 0.05,
    ('Murder', True): 0.1,
    ('Murder', False): 0.9
})

In [ ]:
# P(Findings| Child 1 Death Cause, Child 2 Death Cause)
sallybn.P['Findings'] = pd.Series({
    ('SIDS', 'SIDS', 'Both Murdered'): 0.0,
    ('SIDS', 'SIDS', 'Either Murdered'): 0.0,
    ('SIDS', 'SIDS', 'Neither Murdered'): 1.0,
    ('SIDS', 'Murder', 'Both Murdered'): 0.0,
    ('SIDS', 'Murder', 'Either Murdered'): 1.0,
    ('SIDS', 'Murder', 'Neither Murdered'): 0.0,
    ('Murder', 'SIDS', 'Both Murdered'): 0.0,
    ('Murder', 'SIDS', 'Either Murdered'): 1.0,
    ('Murder', 'SIDS', 'Neither Murdered'): 0.0,
    ('Murder', 'Murder', 'Both Murdered'): 1.0,
    ('Murder', 'Murder', 'Either Murdered'): 0.0,
    ('Murder', 'Murder', 'Neither Murdered'): 0.0
})

# P(Guilty|Findings)
sallybn.P['Guilty'] = pd.Series({
    ('Both Murdered', True): 1.0,
    ('Both Murdered', False): 0.0,
    ('Either Murdered', True): 1.0,
    ('Either Murdered', False): 0.0,
    ('Neither Murdered', True): 0.0,
    ('Neither Murdered', False): 1.0,
})
sallybn.prepare()

# Making queries over `sallybn`


Exact inference on whether Sally is guilty without any evidence


$$\begin{align}P(\text{Guilty}) &= \sum_{c_1}\sum_{c_2}\sum_{d_1}\sum_{i_1}\sum_{d_1}\sum_{i_2}\sum_{f} P(c_1, c_2,  d_1, i_1, d_2, i_2, f, G) \\
&= \sum_{c_1}\sum_{c_2}\sum_{d_1}\sum_{i_1}\sum_{d_2}\sum_{i_2}\sum_{f} P(c_1) P(c_2|c_1)  P(d_1|c_1) P(i_1|c_1) P(d_2|c_2)  P(i_2|c_2) P(f|c_1, c_2) P(G|f) \\
&= \sum_{c_1}P(c_1) \sum_{c_2}P(c_2|c_1) \sum_{d_1}P(d_1|c_1) \sum_{i_1}P(i_1|c_1)  \sum_{d_2}P(d_2|c_2)\sum_{i_2}P(i_2|c_2)\sum_{f} P(f|c_1, c_2) P(G|f) \end{align}$$


Enumeration algorithm 
* step 1 enumerate and sum out nuiance/hidden r.v.s 
* step 2 Bayesian network's semantic (factoring property)
* step 3 simplication

In [ ]:
sallybn.query('Guilty', event={})

Another query, conditional on evidence that both children have bruises but also signs of diseases 

$$P(G| \text{Bruising } C_1 = true, \text{Disease } C_1 = true, \text{Bruising } C_2 = true,  \text{Disease } C_2 = true)$$

In [ ]:
sallybn.query('Guilty', event={'Child 1 Bruising': True, 'Child 1 Sign of Disease': True, 'Child 2 Bruising': True,
                               'Child 2 Sign of Disease': True})

Another query, conditional on evidence that both children have bruises but no signs of diseases 

$$P(G| \text{Bruising } C_1 = true, \text{Disease } C_1 = false, \text{Bruising } C_2 = true,  \text{Disease } C_2 = fasle)$$

In [ ]:
sallybn.query('Guilty', event={'Child 1 Bruising': False, 'Child 1 Sign of Disease': True})

# Summary

* Exact inference over BN
* Use software package to construct a BN
* Walk through of the construction process a BN for Sally Clark's case